## Libraries

In [3]:
import requests
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.service import Service as ChromeService
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from scholarly import scholarly
import argparse
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from threading import Thread
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import queue
import undetected_chromedriver as uc
import os, csv, re
from selenium import webdriver
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem

## Parameter

In [9]:
URL_SCHOLAR = '''https://scholar.google.com/scholar?start=0&q=(reinforcement+learning+OR+RL)+AND+(protocol+routing+OR+routing+strategy+OR+routing+algorithm)+AND+(Wireless+Sensor+Network+OR+WSN+OR+Sensor+Network)&hl=id&as_sdt=0,5&as_ylo=2010'''

URL_IEEE = '''
https://ieeexplore.ieee.org/search/searchresult.jsp?queryText=(reinforcement%20learning%20OR%20RL)%20AND%20(protocol%20routing%20OR%20routing%20strategy%20OR%20routing%20algorithm%20OR%20routing)%20AND%20(Wireless%20Sensor%20Network%20OR%20WSN%20OR%20Sensor%20Network)&highlight=true&returnFacets=ALL&returnType=SEARCH&matchPubs=true&ranges=2010_2024_Year
'''

URL_ACM = '''
https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&field1=AllField&text1=%28%22reinforcement+learning%22+OR+%22RL%22%29+AND+%28%22protocol+routing%22+OR+%22routing+strategy%22+OR+%22routing+algorithm%22+OR+%22routing%22%29+AND+%28%22Wireless+Sensor+Network%22+OR+%22WSN%22+OR+%22Sensor+Network%22%29&searchArea%5B0%5D=SeriesKey&operator%5B0%5D=And&text%5B0%5D=&searchArea%5B1%5D=SeriesKey&operator%5B1%5D=And&text%5B1%5D=&EpubDate=&AfterMonth=1&AfterYear=2010&BeforeMonth=&BeforeYear=&pageSize=20&expand=all
'''

URL_ELSEVIER = '''
https://www.sciencedirect.com/search?qs=%28%22reinforcement%20learning%22%20OR%20%22RL%22%29%20AND%20%28%22protocol%20routing%22%20OR%20%22routing%20strategy%22%20OR%20%22routing%20algorithm%22%20OR%20%22routing%22%29%20AND%20%28%22Wireless%20Sensor%20Network%22%20OR%20%22WSN%22%20OR%20%22Sensor%20Network%22%29&date=2010-2024&show=100&offset=0
'''

URL_WILEY = '''
https://onlinelibrary.wiley.com/action/doSearch?AfterYear=2010&AllField=%28%22reinforcement+learning%22+OR+%22RL%22%29+AND+%28%22protocol+routing%22+OR+%22routing+strategy%22+OR+%22routing+algorithm%22+OR+%22routing%22%29+AND+%28%22Wireless+Sensor+Network%22+OR+%22WSN%22+OR+%22Sensor+Network%22%29&BeforeYear=2024&content=articlesChapters&target=default&pageSize=100&startPage=0
'''

SEARCH_QUERY = '''
(reinforcement learning OR RL) AND (protocol routing OR routing strategy OR routing algorithm OR routing) AND (Wireless Sensor Network OR WSN OR Sensor Network)
'''
SEARCH_QUERY2 = '''
("reinforcement learning" OR "RL") AND ("protocol routing" OR "routing strategy" OR "routing algorithm" OR "routing") AND ("Wireless Sensor Network" OR "WSN" OR "Sensor Network")
'''
DIGITAL_LIBRARIES_ACCEPTED = ['ieee', 'Springer', 'ACM', 'Elsevier', 'Wiley', 'Sage']


software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value,
                     OperatingSystem.LINUX.value]
user_agent_rotator = UserAgent(
    software_names=software_names,
    operating_systems=operating_systems,
    limit=100
)
user_agent = user_agent_rotator.get_random_user_agent()

proxy_server_url = "195.252.236.91"
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument(f'--proxy-server={proxy_server_url}')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--ignore-ssl-errors')
chrome_options.add_argument('--disable-gpu')
# chrome_options.add_argument('--incognito')
# chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--start-maximized")
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "normal"

uc_options = uc.ChromeOptions()
uc_options.add_argument("--no-sandbox")
uc_options.add_argument(f'--proxy-server={proxy_server_url}')
uc_options.add_argument('--ignore-certificate-errors')
uc_options.add_argument('--ignore-ssl-errors')
uc_options.add_argument('--disable-gpu')
uc_options.add_argument('--incognito')
uc_options.add_argument("--headless=new")
uc_options.add_argument("--start-maximized")


information_keys = ['title', 'link', 'number_of_citation', 'article_type', 'publisher', 'publication_date', 'abstract', 'keyword']

## IEEE

Work fine, however consume lot computation if using multiple agent

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import argparse
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from threading import Thread
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.proxy import Proxy, ProxyType
import queue
import os, csv, re
from selenium import webdriver
from csv import DictWriter

global FILE_NAME
# user input arguments
parser = argparse.ArgumentParser()
parser.add_argument('--start', type=int, required=True)
parser.add_argument('--end', type=int, required=True)
parser.add_argument('--number_of_agent', type=int, required=True)

# Webdriver Properties
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--ignore-ssl-errors')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--incognito')
chrome_options.add_argument("--start-maximized")
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "normal"

# Journal Scrapping Argument
information_keys = ['title', 'link', 'number_of_citation', 'article_type', 'publisher', 'publication_date', 'abstract', 'keyword']
DIGITAL_LIBRARIES_ACCEPTED = ['ieee', 'Springer', 'ACM', 'Elsevier', 'Wiley', 'Sage']
URL_IEEE = '''https://ieeexplore.ieee.org/search/searchresult.jsp?queryText=(reinforcement%20learning%20OR%20RL)%20AND%20(protocol%20routing%20OR%20routing%20strategy%20OR%20routing%20algorithm%20OR%20routing)%20AND%20(Wireless%20Sensor%20Network%20OR%20WSN%20OR%20Sensor%20Network)&highlight=true&returnFacets=ALL&returnType=SEARCH&matchPubs=true&ranges=2010_2024_Year'''
df = {}



def split_range(_range, parts): #split a range to chunks
    chunk_size = int(len(_range)/parts)
    chunks = [_range[x:x+chunk_size] for x in range(0, len(_range), chunk_size)]
    return chunks

# Collecting title, link, number of citation, article type, and publication date
def getLinkEachPage(links):
    for link in links:
        print(f"Scraping at page: {link[-1]}")
        count = 1
        try:
            driver = webdriver.Chrome(options=chrome_options)
            driver.get(link)
            article_bodies = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CLASS_NAME, "List-results-items"))
            )
            html = driver.page_source
            driver.quit()
            soup = BeautifulSoup(html, 'html.parser')
            df_temp = {}
            for param in information_keys:
                df_temp[param] = []

            # Check all the paper in div body
            for article in soup.find_all('div', class_='List-results-items'):
                title               = article.find('h3', class_='text-md-md-lh').get_text()
                link_article        = 'https://ieeexplore.ieee.org' + article.find('h3', class_='text-md-md-lh').find('a')['href']

                # Get publisher information such as year, article type and publisher
                publisher_information   = article.find('div', class_='publisher-info-container').find_all('span')
                publisher_information   = [i.get_text() for i in publisher_information if '|' not in i.get_text()]
                publication_date        = publisher_information[0][-4:]
                article_type            = 'other'

                for i in publisher_information:
                    if ((i == 'Conference Paper') or (i == 'Journal Article')):
                        article_type = i
                
                citations = [citation.get_text() for citation in article.find('div', class_='description text-base-md-lh').find_all('span')]

                for citation in citations:
                    if 'Papers' in citation:
                        number_of_citation = int(re.search(r'\d+', citation).group())
                    else:
                        number_of_citation = 0
                
                df_temp['title'].append(title)
                df_temp['link'].append(link_article)
                df_temp['number_of_citation'].append(number_of_citation)
                df_temp['article_type'].append(article_type)
                df_temp['publication_date'].append(publication_date)
                df_temp['publisher'].append('IEEE')

            
            for link_article in df_temp['link']:
                driver = webdriver.Chrome(options=chrome_options)
                driver.get(link_article)
                title = WebDriverWait(driver, 30).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="xplMainContentLandmark"]/div/xpl-document-details/div/div[1]/section[2]/div/xpl-document-header/section/div[2]/div/div/div[1]/div/div[1]/h1'))
                )
                button = driver.find_element(By.ID, 'keywords')
                driver.execute_script("arguments[0].scrollIntoView(true);", button)
                button.click()
                button = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="keywords"]/xpl-document-keyword-list/section/div/ul/li[1]/strong')))

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')

                abstract = soup.find_all('div', class_='u-mb-1')
                abstract = abstract[1].get_text()[len('Abstract:')+1:]
                keyword = soup.find_all('li', class_='doc-keywords-list-item')[1]
                keyword = keyword.get_text()[len("Index Terms"):]

                df_temp['abstract'].append(abstract)
                df_temp['keyword'].append(keyword)
                print(f"\tAt page {link[-1]} Paper found: {count} - {title.text}")
                

                driver.close()
                count += 1

            df_temp = pd.DataFrame(df_temp)
            df = pd.read_csv(FILE_NAME)
            df = pd.concat([df, df_temp], ignore_index=True)
            df.to_csv(FILE_NAME, index=False)
            print(df.tail(5))

        except Exception as e:
            print(f"Error at page: {link[-1]} at paper: {count} \n {e}")

if __name__ == '__main__':
    args = parser.parse_args()
    start = time.time()
    if ((args.end-args.start) == 1):
        print(f"\nParsing IEEE Articles single page at: {args.start}") 
        FILE_NAME = str(np.random.randint(0, 100000)) + f'IEEE_page_single_page_{args.start}_only.csv'
    else:
        FILE_NAME = str(np.random.randint(0, 100000)) + f'IEEE_page_{args.start}_{args.end}.csv'
        print(f"\nParsing IEEE Articles from page {args.start}-{args.end}") 

    for param in information_keys:
        df[param] = []

    pd.DataFrame(df).to_csv(FILE_NAME, index=False)

    # Store link for the whole pages
    URL_IEEE_LIST = [f'{URL_IEEE}&pageNumber={i}' for i in range(args.start, args.end)]
    count = 0
    chunks = split_range(URL_IEEE_LIST, args.number_of_agent)
    
    # Creating a thread agent for opening each page.
    thread_workers = []
    for chunk in chunks:
        t = Thread(target=getLinkEachPage, args=(chunk, ))
        thread_workers.append(t)
        t.start()
    
    for t in thread_workers:
        t.join()


## ACM

Improvement from the IEEE scraping is adding audio notification, increase efficiency by using one loop for searching abstract

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import argparse
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from threading import Thread
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import queue
import os, csv, re
from selenium import webdriver
from csv import DictWriter
import winsound
import time
frequency = 1000  # You can adjust this value
duration = 500  # 5000 milliseconds = 5 seconds

# Beep for 5 seconds


global FILE_NAME
# user input arguments
parser = argparse.ArgumentParser()
parser.add_argument('--start', type=int, required=True)
parser.add_argument('--end', type=int, required=True)
parser.add_argument('--number_of_agent', type=int, required=True)

# Webdriver Properties
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--ignore-ssl-errors')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--incognito')
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--start-maximized")
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "normal"

# Journal Scrapping Argument
information_keys = ['title', 'link', 'number_of_citation', 'article_type', 'publisher', 'publication_date', 'abstract', 'keyword']
DIGITAL_LIBRARIES_ACCEPTED = ['ieee', 'Springer', 'ACM', 'Elsevier', 'Wiley', 'Sage']
URL_ACM = '''https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&field1=AllField&text1=%28%22reinforcement+learning%22+OR+%22RL%22%29+AND+%28%22protocol+routing%22+OR+%22routing+strategy%22+OR+%22routing+algorithm%22+OR+%22routing%22%29+AND+%28%22Wireless+Sensor+Network%22+OR+%22WSN%22+OR+%22Sensor+Network%22%29&searchArea%5B0%5D=SeriesKey&operator%5B0%5D=And&text%5B0%5D=&searchArea%5B1%5D=SeriesKey&operator%5B1%5D=And&text%5B1%5D=&EpubDate=&AfterMonth=1&AfterYear=2010&BeforeMonth=&BeforeYear=&pageSize=50&expand=all'''
df = {}



def split_range(_range, parts): #split a range to chunks
    chunk_size = int(len(_range)/parts)
    chunks = [_range[x:x+chunk_size] for x in range(0, len(_range), chunk_size)]
    return chunks

# Collecting title, link, number of citation, article type, and publication date
def getLinkEachPage(links):
    for link in links:
        
        count = 1
    
        driver = webdriver.Chrome(
    executable_path= ChromeDriverManager().install(), options=chrome_options)
        driver.get(link)
        allow_all_cookies_button = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
        )

        # Click the 'Allow all cookies' button
        allow_all_cookies_button.click()
        html = driver.page_source

        driver.quit()
        soup = BeautifulSoup(html, 'html.parser')

        # Check all the paper in div body
        count_article = 0
        for article in soup.find_all('li', class_='search__item issue-item-container'):
            df_temp = {}
            for param in information_keys:
                df_temp[param] = []
            title               = article.find('h5', class_='issue-item__title').get_text()
            link_article        = 'https://dl.acm.org' + article.find('h5', class_='issue-item__title').find('a')['href']

            # Get publisher information such as year, article type and publisher
            publication_date   = int(article.find('div', class_='bookPubDate simple-tooltip__block--b').get_text()[-4:])
            article_type       = article.find('div', class_='issue-heading').get_text()
            number_of_citation = int(article.find('span', class_='citation').get_text())

            df_temp['title'].append(title)
            df_temp['link'].append(link_article)
            df_temp['number_of_citation'].append(number_of_citation)
            df_temp['article_type'].append(article_type)
            df_temp['publication_date'].append(publication_date)
            df_temp['publisher'].append('ACM')

    
            link_article = df_temp['link'][len(df_temp['link'])-1]
            try:
                t = np.random.random()*3
                time.sleep(t)
                start_time = time.time()
                driver = webdriver.Chrome(
    executable_path= ChromeDriverManager().install(), options=chrome_options)
                driver.get(link_article)
                allow_all_cookies_button = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
                )

                # Click the 'Allow all cookies' button
                allow_all_cookies_button.click()
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                abstract = soup.find("div", id="abstracts").get_text()[len("Abstract"):]
                driver.close()

                df_temp['abstract'].append(abstract)
                df_temp['keyword'].append('none')
                
                count += 1
                print(f"{count*(2/100)*100:.0f}%{"-"*50}{time.time()-start_time} s")
            
            # If error or wont load, try to get
            except:
                driver.close()
                try:
                    retrieve_failed = True
                    for i in range(5):
                        print(f"Trying to retrieve iteration: {i}")
                        if retrieve_failed:
                            t = np.random.random()*3
                            time.sleep(t)
                            start_time = time.time()
                            driver = webdriver.Chrome(
    executable_path= ChromeDriverManager().install(), options=chrome_options)
                            driver.get(link_article)
                            allow_all_cookies_button = WebDriverWait(driver, 10).until(
                            EC.visibility_of_element_located((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
                            )

                            # Click the 'Allow all cookies' button
                            allow_all_cookies_button.click()
                            html = driver.page_source
                            soup = BeautifulSoup(html, 'html.parser')
                            abstract = soup.find("div", id="abstracts").get_text()[len("Abstract"):]
                            driver.close()

                            df_temp['abstract'].append(abstract)
                            df_temp['keyword'].append('none')
                            
                            count += 1

                            print(f"\n{"-"*10} Successfuly retrieved at iteration: {i} {"-"*10}")
                            print(f"{count*(2/100)*100:.0f}%{"-"*50}{time.time()-start_time} ms")
                            


                            retrieve_failed = False
                except:
                    print(f"Failed to retrive at iteration-{i}")

            print(f"Scraping at page: {link[-1]} | Article number: {count_article}")
           
            
            count_article += 1

            try:
                print("\t" + "-"*100)
                df_temp = pd.DataFrame(df_temp)
                df = pd.read_csv(FILE_NAME)
                df = pd.concat([df, df_temp], ignore_index=True)
                df.to_csv(FILE_NAME, index=False)
                print(df.tail(5))
                df_temp = ''
                df = ''
            except:
                print(f"Cant input article numner: {count_article} | title: {title}")

if __name__ == '__main__':
    args = parser.parse_args()
    start = time.time()
    if ((args.end-args.start) == 1):
        print(f"\nParsing ACM Articles single page at: {args.start}") 
        FILE_NAME = f'ACM_page_single_page_{args.start}_only_{np.random.randint(10000, 99999)}.csv'
    else:
        FILE_NAME = f'ACM_page_{args.start}_{args.end}_{np.random.randint(10000, 99999)}.csv'
        print(f"\nParsing ACM Articles from page {args.start}-{args.end}") 

    for param in information_keys:
        df[param] = []

    pd.DataFrame(df).to_csv(FILE_NAME, index=False)

    # Store link for the whole pages
    URL_ACM_LIST = [f'{URL_ACM}&startPage={i-1}' for i in range(args.start, args.end)]
    count = 0
    chunks = split_range(URL_ACM_LIST, args.number_of_agent)
    
    # Creating a thread agent for opening each page.
    thread_workers = []
    for chunk in chunks:
        t = Thread(target=getLinkEachPage, args=(chunk, ))
        thread_workers.append(t)
        t.start()
    
    for t in thread_workers:
        t.join()
    
    for i in range(4):
        winsound.Beep(frequency, duration)
        time.sleep(0.05)
        winsound.Beep(frequency//2, duration)
        

    




## Sciencedirect

In [8]:
driver = webdriver.Chrome(
    executable_path= ChromeDriverManager().install(), options=chrome_options)
driver.get(URL_ELSEVIER)
# allow_all_cookies_button = WebDriverWait(driver, 10).until(
#     EC.visibility_of_element_located((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
# )

## Wiley

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import argparse
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from threading import Thread
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import queue
import os, csv, re
from selenium import webdriver
from csv import DictWriter
import winsound
import time
frequency = 1000  # You can adjust this value
duration = 500  # 5000 milliseconds = 5 seconds



global FILE_NAME
# user input arguments
parser = argparse.ArgumentParser()
parser.add_argument('--start', type=int, required=True)
parser.add_argument('--end', type=int, required=True)
parser.add_argument('--number_of_agent', type=int, required=True)

# Webdriver Properties
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--ignore-ssl-errors')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--incognito')
# chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--start-maximized")
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "normal"

# Journal Scrapping Argument
information_keys = ['title', 'link', 'number_of_citation', 'article_type', 'publisher', 'publication_date', 'abstract', 'keyword']
DIGITAL_LIBRARIES_ACCEPTED = ['ieee', 'Springer', 'ACM', 'Elsevier', 'Wiley', 'Sage']

URL_WILEY = f'https://onlinelibrary.wiley.com/action/doSearch?AfterYear=2010&AllField=%28%22reinforcement+learning%22+OR+%22RL%22%29+AND+%28%22protocol+routing%22+OR+%22routing+strategy%22+OR+%22routing+algorithm%22+OR+%22routing%22%29+AND+%28%22Wireless+Sensor+Network%22+OR+%22WSN%22+OR+%22Sensor+Network%22%29&BeforeYear=2024&content=articlesChapters&target=default&pageSize=100'
 
df = {}

def split_range(_range, parts): #split a range to chunks
    chunk_size = int(len(_range)/parts)
    chunks = [_range[x:x+chunk_size] for x in range(0, len(_range), chunk_size)]
    return chunks

def getAllArticles(links):
    soups = []
    for link in links:
        try:
            count = 1
            driver = webdriver.Chrome(
                executable_path= ChromeDriverManager().install(), 
                options=chrome_options
            )
            driver.get(link)
            WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located(
                    (
                        By.ID, "search-result"
                    )
                )
            )
            html = driver.page_source
            driver.close()
            soups.append(BeautifulSoup(html, 'html.parser'))
        except:
            driver.close()
            retrieve_failed = True
            i = 1
            while ((retrieve_failed) and i<4):
                try:
                    print(f"\nTrying to retrieve all articles at iteration: {i}\n")
                    driver = webdriver.Chrome(
                        executable_path= ChromeDriverManager().install(), 
                        options=chrome_options
                    )
                    driver.get(link)
                    WebDriverWait(driver, 10).until(
                        EC.visibility_of_element_located(
                            (
                                By.ID, "search-result"
                            )
                        )
                    )
                    html = driver.page_source
                    driver.close()
                    retrieve_failed = False
                    print(f"Successfully retrieve all articles at iteration: {i}")
                    soups.append(BeautifulSoup(html, 'html.parser'))
                except:
                    driver.close()
                    print(f"Fail to retrieve all articles at iteration: {i}")
                
                i+=1
    return soups

def getInfoArticle(soups):
    if len(soups) == 0:
        print(f"Failed to retrieve anything")
        return None
    else:
        for soup in soups:
            count_article = 0
            for article in soup.find_all('li', class_='clearfix separator search__item bulkDownloadWrapper'):
                t = np.random.random()*3
                time.sleep(t)
                start_time = time.time()

                title = article.find('h2').find('div').find('input')['title'][len('Select article for bulk download or export: '):]
                link_article = 'https://onlinelibrary.wiley.com' + article.find('h2').find('a')['href']
                publication_date = article.find('div', class_='meta__info hlFld-Title').find('div', class_='meta__details').find('p').get_text()[-4:]
                article_type = article.find('div', class_='meta__header').find('span').get_text()

                try:
                    count = 1
                    driver = webdriver.Chrome(
                        executable_path= ChromeDriverManager().install(), 
                        options=chrome_options
                    )
                    driver.get(link_article)
                    accept_all_button = WebDriverWait(driver, 10).until(
                        EC.visibility_of_element_located(
                            (
                                By.XPATH, "//button[contains(@class, 'osano-cm-accept-all')]"
                            )
                        )
                    )
                    accept_all_button.click()

                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    driver.close()
                except:
                    driver.close()
                    retrieve_failed = True
                    i = 1
                    while ((retrieve_failed) and i<4):
                        try:
                            print(f"\nTrying to retrieve iteration: {i}\n")
                            driver = webdriver.Chrome(
                                executable_path= ChromeDriverManager().install(), 
                                options=chrome_options
                            )
                            driver.get(link_article)
                            accept_all_button = WebDriverWait(driver, 10).until(
                                EC.visibility_of_element_located(
                                    (
                                        By.XPATH, "//button[contains(@class, 'osano-cm-accept-all')]"
                                    )
                                )
                            )
                            accept_all_button.click()

                            html = driver.page_source
                            soup = BeautifulSoup(html, 'html.parser')
                            driver.close()
                            retrieve_failed = False
                            print(f"Successfully retrieve at itration: {i}")
                            
                        except:
                            driver.close()
                            print(f"Fail to retrieve at iteration: {i}")
                        
                        i+=1


                try:
                    abstract = soup.find('div', class_='article-section__content en main').find('p').get_text()

                    print(soup.find('div', class_='article-section__content en main'))

                    number_of_citation = 'none' if soup.find('div', class_='epub-section cited-by-count') == None else soup.find('div', class_='epub-section cited-by-count').find('a').get_text()
                    df_temp = {}
                    for param in information_keys:
                        df_temp[param] = []

                    df_temp['title'].append(title)
                    df_temp['link'].append(link_article)
                    df_temp['number_of_citation'].append(number_of_citation)
                    df_temp['article_type'].append(article_type)
                    df_temp['publication_date'].append(publication_date)
                    df_temp['publisher'].append('Wiley')
                    df_temp['abstract'].append(abstract)
                    df_temp['keyword'].append('none')
                    print("\t" + "-"*100)
                    df_temp = pd.DataFrame(df_temp)
                    df = pd.read_csv(FILE_NAME)
                    count = len(df.index)
                    df = pd.concat([df, df_temp], ignore_index=True)
                    df.to_csv(FILE_NAME, index=False)
                    print(df.tail(5))
                    df_temp = ''
                    df = ''
                    print(f"{count*(2/100)*100:.0f}%{"-"*50}{time.time()-start_time}")
                except:
                    print(f"Cant input article numner: {count_article} | title: {title}")
                


def main(links):
    soup = getAllArticles(links)
    getInfoArticle(soup)
    for i in range(4):
        winsound.Beep(frequency, duration)
        time.sleep(0.05)
        winsound.Beep(frequency//2, duration)

if __name__ == '__main__':
    args = parser.parse_args()
    start = time.time()
    if ((args.end-args.start) == 1):
        print(f"\nParsing Wiley Articles single page at: {args.start}") 
        FILE_NAME = f'Wiley_page_single_page_{args.start}_only_{np.random.randint(10000, 99999)}.csv'
    else:
        FILE_NAME = f'Wiley_page_{args.start}_{args.end}_{np.random.randint(10000, 99999)}.csv'
        print(f"\nParsing Wiley Articles from page {args.start}-{args.end}")
    
    for param in information_keys:
        df[param] = []

    pd.DataFrame(df).to_csv(FILE_NAME, index=False)
    URL_WILEY_LIST = [f'{URL_WILEY}&startPage={i-1}' for i in range(args.start, args.end)]

    count = 0
    chunks = split_range(URL_WILEY_LIST, args.number_of_agent)
    
    # Creating a thread agent for opening each page.
    thread_workers = []
    for chunk in chunks:
        t = Thread(target=main, args=(chunk, ))
        thread_workers.append(t)
        t.start()
    
    for t in thread_workers:
        t.join()
    
    

## Springer

In [261]:
URL_SPRINGER = '''https://link.springer.com/search/page/1?query=%28%22reinforcement+learning%22+OR+%22RL%22%29+AND+%28%22protocol+routing%22+OR+%22routing+strategy%22+OR+%22routing+algorithm%22+OR+%22routing%22%29+AND+%28%22Wireless+Sensor+Network%22+OR+%22WSN%22+OR+%22Sensor+Network%22%29&facet-content-type=%22Article%22'''


driver = webdriver.Chrome(
    executable_path= ChromeDriverManager().install(), 
    options=chrome_options
)
driver.get(URL_SPRINGER)
allow_cookies_button = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located(
        (
            By.CSS_SELECTOR, "button[data-cc-action='accept']"
        )
    )
)
allow_cookies_button.click()
html = driver.page_source
driver.close()
soup = BeautifulSoup(html, 'html.parser')

In [263]:

for article in soup.find('ol', id='results-list').find_all('li'):
    title = article.find('h2').get_text()[1:-1]
    article_type = article.find('p', class_='content-type').get_text(strip=True)
    link_article = 'https://link.springer.com' + article.find('h2').find('a')['href']
    publication_date   = int(article.find('span', class_='year').get_text()[-5:-1])
    
    print(f'Title: {title}')
    print(f'Link: {link_article}')
    print(f'Publication_date: {publication_date}')
    print(f'article_type: {article_type}')
    # print(f'\t\tPublisher: {publisher[-1]}')
    # print(f'\t\tSnippet: {snippet}')
    # print(f'\t\tCitations: {citations}')
    print('-' * 80)


Title: Optimizing IoT-enabled WSN routing strategies using whale optimization-driven multi-criterion correlation approach employs the reinforcement learning agent
Link: https://link.springer.com/article/10.1007/s11082-023-06269-4
Publication_date: 2024
article_type: Article
--------------------------------------------------------------------------------
Title: An intelligent fault detection approach based on reinforcement learning system in wireless sensor network
Link: https://link.springer.com/article/10.1007/s11227-021-04001-1
Publication_date: 2022
article_type: Article
--------------------------------------------------------------------------------
Title: A novel algorithm for wireless sensor network routing protocols based on reinforcement learning
Link: https://link.springer.com/article/10.1007/s13198-021-01414-2
Publication_date: 2022
article_type: Article
--------------------------------------------------------------------------------
Title: Exploring machine learning solution

In [258]:
URL_SPRINGER = '''https://link.springer.com/article/10.1007/s10586-023-04218-0'''


driver = webdriver.Chrome(
    executable_path= ChromeDriverManager().install(), 
    options=chrome_options
)
driver.get(URL_SPRINGER)
allow_cookies_button = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located(
        (
            By.CSS_SELECTOR, "button[data-cc-action='accept']"
        )
    )
)
allow_cookies_button.click()
html = driver.page_source
driver.close()

In [264]:
soup2 = BeautifulSoup(html, 'html.parser')
number_of_citation = soup2.find_all('p', class_="app-article-metrics-bar__count")
if number_of_citation != None:
    if (len(number_of_citation) > 1):
        number_of_citation = soup2.find_all('p', class_="app-article-metrics-bar__count")[1].get_text(strip=True)
        if 'Citations' in number_of_citation:
            number_of_citation = int(re.search(r'\d+', number_of_citation).group())
    else:
        number_of_citation = 'none'
else:
    number_of_citation = 'none'
    
abstract = soup2.find('div', class_="c-article-section__content").get_text()
keyword = soup2.find('ul', class_="c-article-subject-list").find_all('li')
keyword = [f"{i.get_text()}" for i in keyword]
keyword = ','.join(keyword)

print(number_of_citation)
print(abstract)
print(keyword)

AttributeError: 'NoneType' object has no attribute 'get_text'

In [254]:
print(len(number_of_citation) >= 1
)

True


## Scholar

Google scholar tidak dapat bekerja karena masalah captcha

In [ ]:
from fake_useragent import UserAgent

ua=UserAgent()
headers = {'User-Agent': ua.random,
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'en-US,en;q=0.8',
      'Connection': 'keep-alive'}

response = requests.get(URL_SCHOLAR, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

next_links = soup.find_all('div', id='gs_nml')
print(soup.prettify())
next_links = next_links[0].find_all('a')

df_IEEE_links = {}

# Untuk semua page
for link in next_links:
    response = requests.get(URL_SCHOLAR, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    next_links = soup.find_all('div', id='gs_nml')
    next_links = next_links[0].find_all('a')
    articles = soup.find_all('div', class_='gs_r gs_or gs_scl')

    for article in articles:
        title_tag = article.find('h3', class_='gs_rt')
        if title_tag:
            title = title_tag.get_text()
            article_links = title_tag.find('a')['href'] if title_tag.find('a') else 'No link'

        author_info_tag = article.find('div', class_='gs_a')
        if author_info_tag:
            author_info = author_info_tag.get_text()

        snippet_tag = article.find('div', class_='gs_rs')
        if snippet_tag:
            snippet = snippet_tag.get_text()

        citation_tag = article.find('div', class_='gs_fl')
        if citation_tag:
            citation_info = citation_tag.find_all('a')
            if citation_info and len(citation_info) > 2:
                citations = citation_info[2].get_text(strip=True)
            else:
                citations = 'No citations'

        publisher = author_info.replace('\xa0-', ',').split(',')
        print(f'\t\tTitle: {title}')
        print(f'\t\tLink: {article_links}')
        print(f'\t\tAuthor Info: {author_info}')
        print(f'\t\tPublisher: {publisher[-1]}')
        print(f'\t\tSnippet: {snippet}')
        print(f'\t\tCitations: {citations}')
        print('\t\t-' * 80)
    
    
    URL_SCHOLAR = 'https://scholar.google.com' + link['href']
    print(f'Page {link.get_text()} : {'\t\t-' * 80}')


## Join All data

In [155]:
PATH_LIST = []
count = 0
for folder in os.listdir():
    if os.path.isdir(folder):
        for data in os.listdir(folder):
            PATH_LIST.append("./" + folder + '/' + data)

for data in PATH_LIST:
    if count > 1:
        df_temp = pd.read_csv(data)
        df = pd.concat([df, df_temp])
    else:
        df = pd.read_csv(data)
    count += 1
df = df.reset_index(drop=True)
df.to_csv('ALL_ARTICLES.csv', index=False)